In [0]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [0]:
image_size = [224,224] #As VGG16 requires this size of images only

In [0]:
train_path = 'datasets/train'
test_path = 'datasets/test'

In [7]:
vgg = VGG16(input_shape=image_size+[3], weights = 'imagenet', include_top = False)

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
for layer in vgg.layers:
  layer.trainable = False

In [0]:
folders = glob('datasets/train/*')

In [0]:
x = Flatten()(vgg.output)

prediction = Dense(len(folders),activation='softmax')(x)

model = Model(inputs = vgg.input, output = prediction)

In [0]:
model.summary()

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

trai_set = train_datagen.flow_from_directory('datasets/train',
                                                 target_size=(224,224),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('datasets/test',
                                                 target_size=(224,224),
                                                 batch_size=32,
                                                 class_mode='categorical')

r = model.fit_generator(
    train_set,
    validation_data = test_set,
    epochs=5,
    steps_per_epoch = len(train_set),
    validation_steps = len(test_set)
)

In [0]:
plt.plot(r.history['loss'], lable='train_loss')
plt.plot(r.history['val_loss'], lable='val_loss')
plt.legend()
plt.show()

In [0]:
plt.plot(r.history['acc'], lable='train_accuracy')
plt.plot(r.history['val_acc'], lable='val_accuracy')
plt.legend()
plt.show()

In [0]:
from keras.models import load_model

model.save('face_features.h5')